In [14]:
!cv2 version

/bin/bash: cv2: command not found


In [17]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator # Generate batches of tensor image data with real-time data augmentation.
from tensorflow.keras.applications import MobileNetV2 # CNN architecture that works well w/mobile devices as well
from tensorflow.keras.layers import AveragePooling2D, Dropout, Flatten, Dense, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam # stochastic gradient descent method that is based on adaptive estimation of first-order and second-order moments.
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input # Preprocesses a tensor or Numpy array encoding a batch of images.
from tensorflow.keras.preprocessing.image import img_to_array, load_img 
from tensorflow.keras.utils import to_categorical # converts a class vector (integers) to a binary class matrix
from sklearn.preprocessing import LabelBinarizer, MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
# from imutils import paths

import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')
plt.rcParams.update({'font.size': 16})
plt.rcParams['savefig.dpi'] = 200
plt.rcParams['axes.facecolor'] = 'white'
plt.rcParams['grid.color'] = 'lightgrey'

## Preprocessing the data

In [5]:
# init the initial learning rate, number of epochs to train for, and batch size
init_lr = 1e-4
epochs = 20
bs = 32

In [6]:
DIRECTORY = r"data"
CATEGORIES = ["with_mask","without_mask","mask_weared_incorrect"]

In [7]:
print("[INFO] loading images...")

[INFO] loading images...


In [8]:
# get the list of images in our dataset directory, 
# then init the list of data and class images
data = []
labels = []

for category in CATEGORIES:
    path = os.path.join(DIRECTORY, category)
    # looping over image paths and prepocessing the images
    for img in os.listdir(path):
        img_path = os.path.join(path,img)
        image = load_img(img_path,target_size=(175,175)) # resize all images, target size
        image = img_to_array(image)
        image = preprocess_input(image)
        data.append(image)
        labels.append(category)

In [9]:
# labels

In [10]:
# one-hot encoding on the labels (1 and 0) binary format
mlb = MultiLabelBinarizer()
labels = mlb.fit_transform(labels)
# labels = to_categorical(labels)

In [11]:
labels

array([[1, 1, 0, ..., 1, 0, 1],
       [1, 1, 0, ..., 1, 0, 1],
       [1, 1, 0, ..., 1, 0, 1],
       ...,
       [1, 1, 1, ..., 1, 0, 1],
       [1, 1, 1, ..., 1, 0, 1],
       [1, 1, 1, ..., 1, 0, 1]])

In [12]:
# convert data to arrays b/c hidden layers only accept this format
data = np.array(data)
labels = np.array(labels)

In [13]:
data.shape

(8982, 175, 175, 3)

In [14]:
# split the data
X_train, y_train, X_test, y_test = train_test_split(data, labels, test_size=0.2, stratify=labels, random_state=42)

In [15]:
# construct the training image generator for data augmentation
# recreating the images

aug = ImageDataGenerator(
        rotation_range=20,
        zoom_range=0.15,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.15,
        horizontal_flip=True,
        fill_mode="nearest")

In [20]:
# load MobileNetV2 network, ensuring the head FC (Fully Connected) laery sets are left off
baseModel = MobileNetV2(weights="imagenet", include_top=False, input_tensor=Input(shape=(175,175,3)))

2021-08-23 03:12:11.791864: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


9412608/9406464 [==============================] - 0s 0us/step


In [28]:
# create the head of the model that will be placed on top of the base model
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(6,6))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(128, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(2, activation="softmax")(headModel)

In [29]:
# place the head FC model on top of the base model (this will become the actual model we will train)
model = Model(inputs=baseModel.input, outputs=headModel)

# lopp over all layers in the base model and freeze them so they will not be updated during the first training process
for layer in baseModel.layers:
    layer.trainable=False

In [32]:
# compile model
print("[INFO] compiling model...")
opt = Adam(lr=init_lr,decay=init_lr / epochs)
model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])

[INFO] compiling model...
